# Database Building | "The role of sufficiency measures in a decarbonizing Europe"

This notebook is the **first step** to replicate the results from the article:

> **"The role of sufficiency measures in a decarbonizing Europe"**  
> Published: 29 April 2025, Ecological Economics  
> DOI: [10.1016/j.ecolecon.2025.108645](https://doi.org/10.1016/j.ecolecon.2025.108645)

---

## About the Zenodo Repository

The Zenodo repository contains all data and results related to the study.  
**[Zenodo Link](https://zenodo.org/records/15261005)**

### Contents

- **Extended results:** Full set of results, including detailed outputs, scenario comparisons, and sensitivity analyses.
- **Exiobase v3.3.18 - MARIO format:** The unmodified Exiobase v3.3.18 database, converted for MARIO.
- **MARIO database:** Ready-to-use database for background and sufficiency scenario analyses.
- **Files for shocks:** Data and assumptions for scenario-based shocks.

**All files should be used with the instructions and scripts in this GitHub repository, as referenced in the paper.**

---

## Step 1: Download and Extract Exiobase Database

The following cell downloads the Exiobase v3.3.18 database in MARIO format from Zenodo and extracts it for further processing.

In [ ]:
import requests
import os
import zipfile

url = "https://zenodo.org/records/15308822/files/Exiobase%20v3.3.18%20-%20MARIO%20format.zip?download=1" 
output_path = "Data/Exiobase v3.3.18 - MARIO format.zip"
extract_path = "Data/Exiobase v3.3.18 - MARIO format"

os.makedirs(os.path.dirname(output_path), exist_ok=True)
response = requests.get(url, stream=True)
with open(output_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# Extract the ZIP file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    
    # Delete the ZIP file after extraction
    os.remove(output_path)


## Step 2: Parse Exiobase Database with MARIO

This step parses the extracted Exiobase database using the `mario` package, preparing it for aggregation and further modifications.

In [ ]:
import mario
import warnings 
warnings.filterwarnings("ignore") # Pandas became a bit noisy with warnings

db = mario.parse_from_txt(
    path = extract_path+"/flows",
    table = 'SUT', 
    mode ='flows'
)

## Step 3: Aggregate Database

Aggregate the parsed database using the specified aggregation file. This step prepares the data for the addition of new supply chains and further analysis.

In [ ]:
os.makedirs("Data/Database modifications", exist_ok=True)

db.aggregate(
    io = f"Data/Database modifications/Aggregation.xlsx", 
    ignore_nan=True, 
)

## Step 4: Add New Supply Chains

This step adds new supply chains and sectors to the database using custom mapping and value files.  
This step is supported by an ad-hoc function which is soon to be released within the source code of MARIO. Currently, it is available just in the "dev" branch of the MARIO GitHub repository: [https://github.com/it-is-me-mario/MARIO/tree/dev](https://github.com/it-is-me-mario/MARIO/tree/dev)

In [ ]:
os.makedirs("Data/Database modifications", exist_ok=True)

# Comment if the file is ready
# db.get_add_sectors_excel("Data/Database modifications/Add sectors.xlsx") 

# Comment if inventories are already prepared
# db.read_add_sectors_excel(                                               
#     "Data/Database modifications/Add sectors.xlsx", 
#     get_inventories=True, 
# )

# Read inventories from add sectors file
db.read_add_sectors_excel(                                                
    "Data/Database modifications/Add sectors.xlsx", 
    read_inventories=True, 
)

# Add inventories to the database
db.add_sectors()

## Step 5: Add Extra Emissions for Air Transport

This step demonstrates how to add extra emissions for air transport, as explained in Section 3.3.2 of the article.

In [ ]:
import copy
import pandas as pd

# Adding extra emissions for air transport
E_airtransport = copy.deepcopy(db.E.loc['Carbon dioxide, fossil',:])
all_activities = db.get_index('Activity')
all_activities.remove(db.search('Activity', 'Air transport')[0])

E_airtransport.loc[(slice(None),slice(None),all_activities)] = 0
E_airtransport = E_airtransport.to_frame().T
E_airtransport.index = ['Air transport extra emissions']
units_E = pd.DataFrame(['tonnes'], index=['Air transport extra emissions'], columns=['unit'])

db.add_extensions(io=E_airtransport, matrix='E', units=units_E, inplace=True)

## Step 6: Check Footprints of New Activities just added

This step demonstrates how to check on the footprints (CO2 in this example) of the activities added to the database. Consider that all activities aside of diets are initialized as empty and filled in the next step.

In [ ]:
db.f.loc['Carbon dioxide, fossil',('IT',slice(None),db.new_activities)]

## Step 7: Save the Updated Database

This step saves the updated MARIO database, including all modifications and additions, to the specified directory for further use in scenario analyses.


In [ ]:
os.makedirs("Data/MARIO database", exist_ok=True)

db.to_txt(
    path = "Data/MARIO database",
)